In [1]:
import os
import re
import sys
import csv
import time
import json
import itertools

import MeCab
import workFactory

from datetime import datetime
from tqdm import tqdm
from argparse import ArgumentParser
from collections import defaultdict
from collections import OrderedDict

ModuleNotFoundError: No module named 'workFactory'

In [39]:
DATA_DIR = f'./kakuyomu-data/'
SAVE_DIR = f'./morpheme/grand/test'

In [4]:
works = workFactory.read_cleaned_works(f"{DATA_DIR}", 512)

117735405488A-1.jsonl read lines: 9585


100%|█████████████████████████████████████████████████████████████████████████████| 9585/9585 [00:06<00:00, 1542.16it/s]


117735405488B-1.jsonl read lines: 6480


100%|█████████████████████████████████████████████████████████████████████████████| 6480/6480 [00:04<00:00, 1484.17it/s]


117735405488C-1.jsonl read lines: 8156


...

1681692786-1.jsonl read lines: 171


100%|███████████████████████████████████████████████████████████████████████████████| 171/171 [00:00<00:00, 2721.78it/s]


4852201425-1.jsonl read lines: 1087


100%|█████████████████████████████████████████████████████████████████████████████| 1087/1087 [00:01<00:00, 1007.23it/s]

Time: 154.71982288360596


In [7]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [14]:
works = works[:1000]

In [20]:
test_w = works[100]
print(test_w)

{'id': '1177354054880215079', 'title': 'その平面を力に変えて～かつて『ムネナシ』と呼ばれた少女は己が薄胸により最強の力を得る～', 'content': '「どうして……どうしてこうなるのですか？」若き助手は困惑していた。「どうして、だと？聡明な君らしくもない。資料の通りだ。『』１本辺りの接触範囲を最大限にし、以て効果を高める。理に適っておろう？」「確かに、そうかもしれませんが……これでは……これでは、大きければ大きいほど、効果が高まる――ということに、なりませんか？」教授の示した理論を理解するにつれ、込み上げてくる感情があった。「結果的にそうなるが、何か問題があるのかね？先ずは現状の限定された効果対象にとって、最も効果を上げることを考えておるのだ。大きさと効果が比例することに、特に不自然さはなかろう？」その答えが、助手の中の譲れない何かを刺激し、淀み渦巻いていた感情が遂に決壊した。「『』の応用理論を、このような大きさ至上主義的な内容とするべきではありませんっ！」助手は資料を作業机に叩き付け、勢いのまま強く異を唱える。「……ふむ、我輩の理論を否定するのであれば、君の理論を示したまえ。そこまで言うからには、既に君にも何かしら構想があるのだろう？」助手の剣幕に若干の当惑を示しながらも、教授は冷静に促す。理論には理論で対抗せよ。一人の研究者として、助手を扱えばこその提', 'labels': ['40'], 'wakati': ''}


In [61]:
cnt = test_w['content']
sep = re.split("。|」|？|！",cnt)
print(sep)
mecab = MeCab.Tagger("")

s_list = []
noun_list = []
verb_list = []
adjv_list = []


for sent in sep:
    node = mecab.parseToNode(sent)
    surface = []
    while node:
        surface.append(node.surface)
        term = node.feature.split(",")[0]
        if term == "名詞":
            noun_list.append(node.surface)
        elif term == "動詞":
            verb_list.append(node.surface)
        elif term == "形容詞" or term == "副詞":
            adjv_list.append(node.surface)
        else:
            pass
        node = node.next
    s_list.append(' '.join(surface))

print(noun_list)
print(s_list)
with open(f'./morpheme/grand/test', 'w', encoding='utf-8') as f:
        for line in s_list:
            f.write(f'{line}\n')

['「どうして……どうしてこうなるのですか', '', '若き助手は困惑していた', '「どうして、だと', '聡明な君らしくもない', '資料の通りだ', '『』１本辺りの接触範囲を最大限にし、以て効果を高める', '理に適っておろう', '', '「確かに、そうかもしれませんが……これでは……これでは、大きければ大きいほど、効果が高まる――ということに、なりませんか', '', '教授の示した理論を理解するにつれ、込み上げてくる感情があった', '「結果的にそうなるが、何か問題があるのかね', '先ずは現状の限定された効果対象にとって、最も効果を上げることを考えておるのだ', '大きさと効果が比例することに、特に不自然さはなかろう', '', 'その答えが、助手の中の譲れない何かを刺激し、淀み渦巻いていた感情が遂に決壊した', '「『』の応用理論を、このような大きさ至上主義的な内容とするべきではありませんっ', '', '助手は資料を作業机に叩き付け、勢いのまま強く異を唱える', '「……ふむ、我輩の理論を否定するのであれば、君の理論を示したまえ', 'そこまで言うからには、既に君にも何かしら構想があるのだろう', '', '助手の剣幕に若干の当惑を示しながらも、教授は冷静に促す', '理論には理論で対抗せよ', '一人の研究者として、助手を扱えばこその提']
['助手', '困惑', '聡明', '資料', '通り', '１', '辺り', '接触', '範囲', '最大', '効果', '理', '効果', 'こと', '教授', '理論', '理解', '感情', '結果', '問題', '現状', '限定', '効果', '対象', '効果', 'こと', '効果', '比例', 'こと', '答え', '助手', '中', '刺激', '淀み', '感情', '決壊', '応用', '理論', '至上', '主義', '内容', '助手', '資料', '作業', '机', '勢い', 'まま', '異', '理論', '否定', '理論', 'まえ', '構想', '助手', '剣幕', '若干', '当惑', '教授', '冷静', '理論', '理論', '対抗', '一人', '研究', '助手', '提']
[' 「 どう し て … … ど

In [72]:
def analysis(n, v, a, path, work):
    noun_list = []
    verb_list = []
    adjv_list = []
    surface_list =[]
    
    cnt = work['content']
    sep = re.split("。|」|？|！", cnt)
    
    mecab = MeCab.Tagger("")
    for sent in sep:
        node = mecab.parseToNode(sent)
        surface = []
        while node:
            surface.append(node.surface)
            term = node.feature.split(",")[0]
            if term == "名詞":
                noun_list.append(node.surface)
            elif term == "動詞":
                verb_list.append(node.surface)
            elif term == "形容詞" or term == "副詞":
                adjv_list.append(node.surface)
            else:
                pass
            node = node.next
        surface_list.append(' '.join(surface))
    
    os.makedirs(path, exist_ok=True)
    with open(f'{path}/{work["id"]}', 'w', encoding='utf-8') as f:
        for line in surface_list:
            f.write(f'{line}\n')
            
    for noun in noun_list:
        n[noun] += 1
            
    for verb in verb_list:
        v[verb] += 1
            
    for adjv in adjv_list:
        a[adjv] += 1
    
    return n, v, a

In [74]:
i = 0
for work in batch(works, 200):
    print(f'batch={i}')
    
    noun = defaultdict(int)
    verb = defaultdict(int)
    adjv = defaultdict(int)
    
    for w in tqdm(work):
        noun, verb, adjv = analysis(noun, verb, adjv, f'{SAVE_DIR}/{i}', w)
    
    noun = OrderedDict(sorted(noun.items(), key=lambda x:x[1], reverse=True))
    verb = OrderedDict(sorted(verb.items(), key=lambda x:x[1], reverse=True))
    adjv = OrderedDict(sorted(adjv.items(), key=lambda x:x[1], reverse=True))
    
    os.makedirs(SAVE_DIR, exist_ok=True)
    with open(f'{SAVE_DIR}/{i}/noun.tsv', 'w') as f:
        w = csv.writer(f, delimiter='\t')
        w.writerow(['word', 'count'])
        for key, val in noun.items():
            w.writerow([key, val])
    
    with open(f'{SAVE_DIR}/{i}/verb.tsv', 'w') as f:
        w = csv.writer(f, delimiter='\t')
        w.writerow(['word', 'count'])
        for key, val in verb.items():
            w.writerow([key, val])
            
    with open(f'{SAVE_DIR}/{i}/adjv.tsv', 'w') as f:
        w = csv.writer(f, delimiter='\t')
        w.writerow(['word', 'count'])
        for key, val in adjv.items():
            w.writerow([key, val])
    i+=1

batch=0


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 93.82it/s]


batch=1


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 94.53it/s]


batch=2


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 94.39it/s]


batch=3


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 94.23it/s]


batch=4


100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 94.46it/s]


In [3]:
import MeCab
txt = '妖力 と 妖術 と いう 二 つ の 能力 が ある の 妖力 と 妖術 凱 斗 に は 正直 どちら も 同じ に 聞こえ た
一体 何 が どう 違う と いう の か 妖術 は 呪文 みたい な もの ね 手順 を 踏む こと で 発動 する もの を
指す の 一方 の 妖力 、 これ は 常時 発動 し て いる 妖魅 特有 の 力 の こと よ 例えば >、 と 言っ て 穂積
は 人差し 指 を 立て た 幽霊 、 って いう の は 身体 的 特徴 と し て 『 実態 が ない 』 し 『 半 透明 』
だっ たり する わ よ ね 一 >反 木綿 だっ たら 『 飛行 』 できる と か こう いっ た 常態 的 に 発揮 さ れ 
て いる 能力 を "" 妖力 "" と 呼ぶ の 立て て い た 指 で 赤い 縁 の 眼鏡 を 押し上げ 穂積 は 続ける それ
と は 別 に 、 狸 が 『 化ける 』 だ と か 、 セイ レーン が その 歌声 で 『 魅了 』 する だ と か 、 雷神
が 『 雷撃 』 を 放っ たり と か 能力 を 発動 する ため に プロセス や 集中 など の 作業 が 必要 な もの 
を 総称 し て "" 妖術 "" と 呼ん で いる の さっき 貴方 が 出し た 火柱 も 妖術 に 分類 さ れる わ 凱 斗 
は 自分 の 手 を 見る 指先 から 燃え上がっ た 炎 の 柱 の こと を 思い出す 確か に 燃えろ と 念じ た 途端 
に 起>こっ た 出来事 だっ た 貴方 の 場合 、 古 籠 火 の 骸 露 を 取り入れ た こと で 火 に 関する 妖術 を
扱える よう に なっ て いる って わけ 炎 の 妖術 は 推し 並べ て 強力 な 物 が 多い から 、 ここ で 扱い 
方 を 学ん で おく と 後々 役 に 立つ わ 断定 し 、 さあ やっ て み て と ばかり に 穂積 は 顎 を"
mecab = MeCab.Tagger("-Owakati")
mecab.parse('ラベルと形態素分析した名詞のリストをそれぞれWord2Vecで変換する')

'ラベル と 形態 素 分析 し た 名詞 の リスト を それぞれ Word 2 Vec で 変換 する \n'